# 좌표계 변환

In [1]:
import pandas as pd
import numpy as np
import pyproj
import folium

In [2]:
df = pd.read_csv("C:/tobeO_V220414/tobeO_V220414/Data_02.ShadeCanopy/횡단보도/서울시 교통안전시설물 횡단보도 정보.csv",
                 encoding='cp949'
                )

C:\Users\user\AppData\Local\Temp\ipykernel_7296\882766023.py:1: DtypeWarning: Columns (18,19,20,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/tobeO_V220414/tobeO_V220414/Data_02.ShadeCanopy/횡단보도/서울시 교통안전시설물 횡단보도 정보.csv",


## 데이터 타입을 변환한 후 이상값들을 제거

In [3]:
df['경도'] = pd.to_numeric(df['경도'], errors="coerce") # 경로 컬럼
df['위도'] = pd.to_numeric(df['위도'], errors="coerce") # 위도 컬럼

df = df.dropna(subset=['경도', '위도']) # 경도, 위도 컬럼 널값 삭제
df.index=range(len(df))
df.head()

,순번,교차로명,상태 (공통),횡단보도종류코드,가로길이,세로길이,화살표시수량,화살표시길이,고가 (공통),구경찰서코드 (공통),...,설치일,교체일,공간데이터,이력ID,공사관리번호,구횡단보도관리번호,공사형태 (공통),교차로코드,경도,위도
0,2,06-0000029829,1.0,3.0,0.0,0.0,0.0,1.0,1.0,190.0,...,NaN,NaN,NaN,87758,2008-0102-558,06-029829,1,NaN,200997.6242,556204.9599
1,3,06-0000030030,1.0,2.0,0.0,0.0,4.0,1.0,1.0,390.0,...,NaN,NaN,NaN,101702,2008-1108-025,06-030030,1,NaN,192679.8948,559453.6699
2,4,06-0000030030,1.0,2.0,0.0,0.0,4.0,1.0,1.0,390.0,...,NaN,NaN,NaN,101701,2008-1108-025,06-030030,1,NaN,192678.3015,559462.8943
3,5,06-0000031955,1.0,3.0,NaN,NaN,4.0,6.0,1.0,330.0,...,20220818,20220818,NaN,73830,2022-0207-027,06-031955,4,1578,187544.4088,544732.0084
4,6,06-0000031956,1.0,3.0,NaN,NaN,4.0,6.0,1.0,330.0,...,20220818,20220818,NaN,73831,2022-0207-027,06-031956,4,1578,187544.9312,544733.1062


## 좌표계 epsg:5186 -> epsg:4326 변경

In [4]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [5]:
# DataFrame -> NumPy Array 변환
coord = np.array(df[['경도', '위도']])
coord

array([[200997.6242, 556204.9599],
       [192679.8948, 559453.6699],
       [192678.3015, 559462.8943],
       ...,
       [192699.6305, 545494.7766],
       [190746.25  , 545409.762 ],
       [192707.8419, 545502.094 ]])

In [6]:
# 좌표계 정보 설정
p1_type = "epsg:5186"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

C:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_7296\2093212355.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = py

array([[127.011298  ,  37.60542377],
       [126.91706805,  37.63466564],
       [126.91704991,  37.63474874],
       ...,
       [126.91743061,  37.50889698],
       [126.89533838,  37.50811349],
       [126.91752341,  37.50896297]])

In [7]:
df['경도'] = result[:, 0]
df['위도'] = result[:, 1]

In [8]:
df.head()

,순번,교차로명,상태 (공통),횡단보도종류코드,가로길이,세로길이,화살표시수량,화살표시길이,고가 (공통),구경찰서코드 (공통),...,설치일,교체일,공간데이터,이력ID,공사관리번호,구횡단보도관리번호,공사형태 (공통),교차로코드,경도,위도
0,2,06-0000029829,1.0,3.0,0.0,0.0,0.0,1.0,1.0,190.0,...,NaN,NaN,NaN,87758,2008-0102-558,06-029829,1,NaN,127.011298,37.605424
1,3,06-0000030030,1.0,2.0,0.0,0.0,4.0,1.0,1.0,390.0,...,NaN,NaN,NaN,101702,2008-1108-025,06-030030,1,NaN,126.917068,37.634666
2,4,06-0000030030,1.0,2.0,0.0,0.0,4.0,1.0,1.0,390.0,...,NaN,NaN,NaN,101701,2008-1108-025,06-030030,1,NaN,126.917050,37.634749
3,5,06-0000031955,1.0,3.0,NaN,NaN,4.0,6.0,1.0,330.0,...,20220818,20220818,NaN,73830,2022-0207-027,06-031955,4,1578,126.859137,37.501969
4,6,06-0000031956,1.0,3.0,NaN,NaN,4.0,6.0,1.0,330.0,...,20220818,20220818,NaN,73831,2022-0207-027,06-031956,4,1578,126.859142,37.501979


## 파일 저장

In [9]:
df.to_csv('./서울시 교통안전시설물 횡단보도 정보_좌표변경.csv', encoding='cp949')